In [178]:
import simpy
import numpy as np
import statistics

process_times = []


In [179]:
class Hospital(object):
    def __init__(self, env, num_nurses, num_radiologists, num_specialists):
        self.env = env
        self.nurse = simpy.Resource(env, num_nurses)
        self.radiologist = simpy.Resource(env, num_radiologists)
        self.specialist = simpy.Resource(env, num_specialists)

    def check_vitals(self, patient):
        yield self.env.timeout(np.random.randint(5, 10))
    
    def take_xray(self, patient):
        yield self.env.timeout(np.random.randint(15, 25))
    
    def consult_patient(self, patient):
        yield self.env.timeout(10, 20)


In [180]:
def go_to_hospital(env, patient, hospital):
    # Patient arrives at the hospital
    arrival_time = env.now

    with hospital.nurse.request() as request:
        yield request
        print(f"Nurse starts checking vitals of patient {patient} at {env.now}")
        yield env.process(hospital.check_vitals(patient))
        print(f"Nurse finished checking patient {patient} at {env.now}")

    with hospital.specialist.request() as request:
        yield request
        print(f"Specialist starts consult with patient {patient} at {env.now}")
        yield env.process(hospital.consult_patient(patient))
        print(f"Specialist finished consult with patient {patient} at {env.now}")

    if np.random.choice([True, False]):
        with hospital.radiologist.request() as request:
            yield request
            print(f"Radiologist takes xray of patient {patient} at {env.now}")
            yield env.process(hospital.take_xray(patient))
            print(f"Radiologist releases patient {patient} at {env.now}")

    # Patient goes home
    finish_time = env.now
    process_time = finish_time - arrival_time
    print(f">> Patient {patient} started at {arrival_time} and left at {finish_time} with total process time {process_time}")
    process_times.append(process_time)


In [181]:
def run_hospital(env, num_nurses, num_radiologists, num_specialists):
    hospital = Hospital(env, num_nurses, num_radiologists, num_specialists)
    
    for patient in range(3):
        env.process(go_to_hospital(env, patient, hospital))

    while True:
        yield env.timeout(0.20)  # Wait a bit before generating a new patient

        patient += 1
        env.process(go_to_hospital(env, patient, hospital))


In [182]:
def calculate_average_time(process_times):
    average_process = statistics.mean(process_times)
    return average_process


In [183]:
def main(params=[1, 1, 1]):
    # Setup
    np.random.seed(42)
    num_nurses, num_radiologists, num_specialists = params

    # Run the simulation
    env = simpy.Environment()
    env.process(run_hospital(env, num_nurses, num_radiologists, num_specialists))
    env.run(until=90)

    # View the results
    average_process = calculate_average_time(process_times)
    print(
        "Running simulation...",
        f"\nThe average process time is {average_process} minutes.",
    )


In [184]:
main()

Nurse starts checking vitals of patient 0 at 0
Nurse finished checking patient 0 at 8
Specialist starts consult with patient 0 at 8
Nurse starts checking vitals of patient 1 at 8
Nurse finished checking patient 1 at 17
Nurse starts checking vitals of patient 2 at 17
Specialist finished consult with patient 0 at 18
>> Patient 0 started at 0 and leaves at 18 with total process time 18
Specialist starts consult with patient 1 at 18
Nurse finished checking patient 2 at 24
Nurse starts checking vitals of patient 3 at 24
Specialist finished consult with patient 1 at 28
Radiologist takes xray of patient 1 at 28
Specialist starts consult with patient 2 at 28
Nurse finished checking patient 3 at 33
Nurse starts checking vitals of patient 4 at 33
Specialist finished consult with patient 2 at 38
Specialist starts consult with patient 3 at 38
Nurse finished checking patient 4 at 39
Nurse starts checking vitals of patient 5 at 39
Nurse finished checking patient 5 at 46
Nurse starts checking vitals 